## Quality assurance of prediction results

In the previous tutorial, we looked at various ways to visualise the results of our model.  

These are useful for quality assurance purposes because they allow us to understand how our model works and verify that it is doing something sensible.  They can also be useful to identify examples where the model is not performing as expected.  

In addition to these spot checks, Splink also has functions to perform more formal accuracy analysis.  These functions allow you to understand the likely prevalence of false positives and false negatives in your linkage models.

They rely on the existence of a sample of labelled (ground truth) matches, which may have been produced (for example) by human beings.  For the accuracy analysis to be unbiased, the sample should be representative of the overall dataset.

In [1]:
# Rerun our predictions to we're ready to view the charts
from splink.duckdb.linker import DuckDBLinker
from splink.datasets import splink_datasets
import altair as alt
alt.renderers.enable('html')

df = splink_datasets.fake_1000
linker = DuckDBLinker(df)
linker.load_model("../demo_settings/saved_model_from_demo.json")
df_predictions = linker.predict(threshold_match_probability=0.2)

## Load in labels

The labels file contains a list of pairwise comparisons which represent matches and non-matches.

The required format of the labels file is described [here](https://moj-analytical-services.github.io/splink/linkerqa.html#splink.linker.Linker.roc_chart_from_labels).

In [2]:
import pandas as pd
df_labels = splink_datasets.fake_1000_labels
df_labels.head(5)
labels_table = linker.register_labels_table(df_labels)

## Receiver operating characteristic curve

A [ROC chart](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) shows how the number of false positives and false negatives varies depending on the match threshold chosen.  The match threshold is the match weight chosen as a cutoff for which pairwise comparisons to accept as matches.



In [3]:
linker.roc_chart_from_labels_table(labels_table)

alt.Chart(...)

### Precision-recall chart

An alternative representation of truth space is called a [precision recall curve](https://stats.stackexchange.com/questions/7207/roc-vs-precision-and-recall-curves).

This can be plotted as follows:

In [4]:
linker.precision_recall_chart_from_labels_table(labels_table)

alt.Chart(...)

## Truth table

Finally, Splink can also report the underlying table used to construct the ROC and precision recall curves.

In [5]:
roc_table = linker.truth_space_table_from_labels_table(labels_table)
roc_table.as_pandas_dataframe(limit=5)

,truth_threshold,match_probability,row_count,p,n,tp,tn,fp,fn,P_rate,N_rate,tp_rate,tn_rate,fp_rate,fn_rate,precision,recall,f1
0,-26.442571,1.096460e-08,1225.0,80.0,1145.0,80.0,0.0,1145.0,0.0,0.065306,0.934694,1.0,0.000000,1.000000,0.0,0.065306,1.0,0.122605
1,-25.337736,2.358204e-08,1225.0,80.0,1145.0,80.0,47.0,1098.0,0.0,0.065306,0.934694,1.0,0.041048,0.958952,0.0,0.067912,1.0,0.127186
2,-24.371460,4.607438e-08,1225.0,80.0,1145.0,80.0,154.0,991.0,0.0,0.065306,0.934694,1.0,0.134498,0.865502,0.0,0.074697,1.0,0.139010
3,-24.370218,4.611406e-08,1225.0,80.0,1145.0,80.0,199.0,946.0,0.0,0.065306,0.934694,1.0,0.173799,0.826201,0.0,0.077973,1.0,0.144665
4,-23.939989,6.213625e-08,1225.0,80.0,1145.0,80.0,230.0,915.0,0.0,0.065306,0.934694,1.0,0.200873,0.799127,0.0,0.080402,1.0,0.148837
